# 第4章 朴素贝叶斯法

> Naive Bayes
> * 分类；
> * = 贝叶斯定理 + 特征条件独立假设；
> * 对于给定的training data，首先基于特征条件独立假设，学习输入/输出的联合概率分布；然后基于此模型，对给定的输入$\mathbf{x}$，利用贝叶斯定理求出后验概率最大的输出$y$。

## 关于贝叶斯定理: 《Think Bayes》

__条件概率__：$P(A|B)$，表示在给定B条件下A事件发生的概率。

__联合概率__：$P(A\ and\ B)$，表示两个事件同时发生的概率，有：

$$P(A\ and\ B) = P(A)P(B|A)$$

当事件A与事件B独立，即$P(B|A) = P(B)$时，有：

$$P(A\ and\ B) = P(A)P(B|A) = P(A)P(B)$$

__贝叶斯定理__：用$P(B|A)$来求$P(A|B)$

$$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$$

__证明__：

$$P(A\ and\ B) = P(B\ and\ A)$$
$$P(A)P(B|A) = P(B)P(A|B)$$
$$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$$

__历时诠释：另一种理解贝叶斯定理的思路__

贝叶斯定理为我们提供了一种根据数据集$D$的内容变化更新假设概率$H$的方法。我们先将贝叶斯定理改写为：

$$P(H|D) = \frac{P(H)P(D|H)}{P(D)}$$

其中：

* $P(H)$为__先验概率__，即得到新数据$D$之前，某一假设成立的概率；
* $P(H|D)$为__后验概率__，即得到新数据$D$之后，某一假设成立的概率；
* $P(D|H)$为__似然度__，即在该假设成立的条件下，得到新数据$D$的概率；
* $P(D)$为__标准化常量__，在任何假设下得到新数据$D$的概率：
    * 首先我们有一个互斥且完备的假设集合(suite)，上面提到的$H$也来自于这个假设集；
    * $P(D) = \sum_{i=1}^{k} p(H_i)p(D|H_i)$

## 4.1 朴素贝叶斯法的学习与分类

训练数据集由$P(X,Y)$独立同分布产生，其中$\mathbf{x_i}\in \mathbf{R}^n$，$y_i \in \{c_1,c_2,\cdots,c_K\}$，$i=1,2,\cdots,N$：

$$T=\{(\mathbf{x_1},y_1),(\mathbf{x_2},y_2),\cdots,(\mathbf{x_N},y_N)\}$$

预测(后验概率)：

$$y = \underset{c_k}{argmax}\ P(Y=c_k|X=\mathbf{x}) = \underset{c_k}{argmax}\ \frac{P(Y=c_k)P(X=\mathbf{x}|Y=c_k)}{P(X=\mathbf{x})} = \underset{c_k}{argmax}\ \frac{P(Y=c_k)P(X=\mathbf{x}|Y=c_k)}{\sum_k P(Y=c_k)P(X=\mathbf{x}|Y=c_k)}$$

由上式知，首先需要学习__先验概率分布__$P(Y=c_k),\quad k=1,2,\cdots,K$；然后学习__似然度(李航:条件概率分布)__$P(X=\mathbf{x}|Y=c_k)=P(X^{(1)}=\mathbf{x}^{(1)},X^{(2)}=\mathbf{x}^{(2)},\cdots,X^{(n)}=\mathbf{x}^{(n)}|Y=c_k),\quad k=1,2,\cdots,K$

假设$X^{(j)}$可取值为$S_j$个，$j=1,2,\cdots,n$，那么参数个数为$K\prod^n_{j=1}S_j$——总参数个数：$K\prod^n_{j=1}S_j+K$——太大了。

此时，运用条件独立性假设：

$$P(X=\mathbf{x}|Y=c_k)=P(X^{(1)}=\mathbf{x}^{(1)},X^{(2)}=\mathbf{x}^{(2)},\cdots,X^{(n)}=\mathbf{x}^{(n)}|Y=c_k)=\prod^n_{j=1}P(X^{(j)}=\mathbf{x}^{(j)}|Y=c_k),\quad k=1,2,\cdots,K$$

总参数个数缩小为：$K\sum^n_{j=1}S_j+K$

回到预测，我们知道，对于任意输入$\mathbf{x}$，其预测为：

$$y = \underset{c_k}{argmax}\ \frac{P(Y=c_k)P(X=\mathbf{x}|Y=c_k)}{\sum_k P(Y=c_k)P(X=\mathbf{x}|Y=c_k)}$$

上式中的分母，对于每个$c_k$的计算时都是相同的，因此只要比分子即可，即：

$$y = \underset{c_k}{argmax}\ P(Y=c_k)P(X=\mathbf{x}|Y=c_k)=\underset{c_k}{argmax}\ P(Y=c_k)\prod^n_{j=1}P(X^{(j)}=\mathbf{x}^{(j)}|Y=c_k)$$


## 4.2 朴素贝叶斯法的参数估计

朴素贝叶斯法，需要估计：
* $P(Y=c_k)$：$K$个参数；
* $P(X^{(j)}=a^{(j)}_l|Y=c_k)$：其中，$j=1,2,\cdots,n$，$l=1,2,\cdots,S_j$，$K\sum^n_{j=1}S_j$个参数。

### 一. 极大似然估计

先验概率的极大似然估计为：

$$P(Y=c_k)=\frac{\sum_{i=1}^N I(y_i=c_k)}{N},\quad k=1,2,\cdots,K$$

似然度的极大似然估计为：

$$P(X^{(j)}=a^{(j)}_l|Y=c_k)=\frac{\sum_{i=1}^N I(\mathbf{x}^{(j)}_i=a^{(j)}_l,y_i=c_k)}{\sum_{i=1}^N I(y_i=c_k)},\quad k=1,2,\cdots,K,\quad j=1,2,\cdots,n,\quad l=1,2,\cdots,S_j$$

### 二. 贝叶斯估计

极大似然估计的一个缺点是，可能会出现所估计的参数值为0的情况：
* 先验概率的估计值为0：某一个类别在训练集中未出现？
* 似然度的估计值为0：某一个类别下某个维度的可能取值未在训练集中出现？

似然度的贝叶斯估计为：$\lambda \ge 0$

$$P_\lambda(X^{(j)}=a^{(j)}_l|Y=c_k)=\frac{\sum_{i=1}^N I(\mathbf{x}^{(j)}_i=a^{(j)}_l,y_i=c_k)+\lambda}{\sum_{i=1}^N I(y_i=c_k)+S_j\lambda},\quad k=1,2,\cdots,K,\quad j=1,2,\cdots,n,\quad l=1,2,\cdots,S_j$$

特别的：

* $\lambda=0$时是极大似然估计；
* $\lambda=1$时是拉普拉斯平滑。

先验概率的贝叶斯估计为：$\lambda \ge 0$

$$P_\lambda(Y=c_k)=\frac{\sum_{i=1}^N I(y_i=c_k)+\lambda}{N+K\lambda},\quad k=1,2,\cdots,K$$